In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [2]:
with open("ideal_counts_hardware.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_hardware.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_hardware.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_hardware.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'0011001110': 200000}
noise est dist:  {'0001001110': 0.05687, '0101000110': 0.00236, '0010101010': 0.002675, '0011011110': 0.04006, '0010001110': 0.031515, '0010000110': 0.00537, '0011101110': 0.022685, '1011001100': 0.001825, '0011100010': 0.002055, '0111001010': 0.00784, '0110011110': 0.00322, '1001000010': 0.00062, '0111001110': 0.02371, '1011101100': 0.00042, '0010111110': 0.00282, '1011001010': 0.00449, '0011001110': 0.107525, '0011001010': 0.033515, '0100001110': 0.004575, '0101001000': 0.00054, '0011101001': 0.000125, '0011001100': 0.010895, '0111000110': 0.00432, '0111101010': 0.00259, '1010001100': 0.000525, '1001011100': 0.00044, '0111011101': 0.000125, '0111011010': 0.00294, '0001001010': 0.01872, '1100001110': 0.00066, '1101001110': 0.001875, '0000001110': 0.01785, '0110000110': 0.00118, '0110001011': 0.000355, '1001001010': 0.002695, '0000100110': 0.000975, '0011000110': 0.018045, '0011010110': 0.006485, '0001011110': 0.021845, '0000011110': 0.00677, '0011011011'

In [3]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [4]:
num_qubits=len(list(noisy_counts.items())[0][0])

# if 2**10<2**num_qubits:
num_keep=2**15#2**10#num_qubits#2**10
if num_keep>2**num_qubits:
    num_keep=2**num_qubits
noisy_counts_orig=deepcopy(noisy_counts)
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}
elem_sorted=sorted(quasi_corrected_counts.items(), key=lambda x: x[1])[::-1]
print(elem_sorted[0:5])



num keep  1024
max exist  1023
min exist  0
size before  1024
1024
1024
[('1111111111', 65274.2315064977), ('0000000000', 13307.9888819245), ('1111111110', 9678.94198785535), ('1000001111', 9629.39701267280), ('1000000111', 9551.25040078268)]


In [5]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, 
               legend=["ideal", "raw", "corrected"], title="GQuantum Phase Estimation 10 Qubits Quantum Hardware")
fig.savefig("plot_qpe10_counts_hardware_ibm_marrakesh.jpg", bbox_inches="tight")


In [6]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, 
                  legend=["ideal", "raw", "corrected"], title="Quantum Phase Estimation 10 Qubits Quantum Hardware") #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.savefig("plot_qpe10_dist_hardware_ibm_marrakesh.jpg", bbox_inches="tight")

In [7]:
from qiskit.quantum_info import hellinger_fidelity
print(hellinger_fidelity(ideal_counts, quasi_corrected_counts))
print(hellinger_fidelity(ideal_counts, noisy_counts_orig))
print(ideal_counts)
print(quasi_corrected_counts)
print(noisy_counts_orig)

0.32637115753248863
0.029350000000000053
{'1111111111': 200000}
{'1111110100': 3.14785150810610, '0111001011': 8.44170772060124, '0100001110': 20.1244505089949, '0000000111': 20.2413712317134, '1000000100': 22.2060206676852, '0111101000': 32.2003544114121, '1000111101': 35.6374265918161, '1011111101': 43.0393005073247, '1100000101': 48.2431279045333, '1000111110': 64.9642272758683, '1000001000': 65.4125496996846, '1110100011': 66.4163992486178, '1100111001': 86.9391052190006, '1011100011': 89.4821405467676, '1110001011': 90.9146974268563, '0111100100': 101.755421295287, '0000000110': 117.813966270817, '0000111000': 123.590654943595, '1110000001': 125.344443887705, '1111010101': 130.791483742279, '0111110111': 165.813938689127, '0000000011': 181.315313164200, '1100000111': 185.643769451704, '1001110011': 186.695422133926, '0100001000': 204.933464293042, '0000001010': 233.080338841876, '0100100000': 258.398131156656, '0110000000': 266.707018405134, '1110101111': 273.402225241587, '100111